<a href="https://colab.research.google.com/github/CarlosRomero55/Remote-Sensing-Satelite-image/blob/main/Remote_Sensing_Satelite_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Librerías necesarias
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.utils import to_categorical

# Configuración para gráficos interactivos
pio.renderers.default = 'colab'  # Usa 'browser' si lo ejecutas localmente en Jupyter

# Rutas de los datasets
train_images_dir = '/content/drive/MyDrive/datasett/train/images'
train_labels_dir = '/content/drive/MyDrive/datasett/train/labels'
valid_images_dir = '/content/drive/MyDrive/datasett/valid/images'
valid_labels_dir = '/content/drive/MyDrive/datasett/valid/labels'
test_images_dir = '/content/drive/MyDrive/datasett/test/images'
test_labels_dir = '/content/drive/MyDrive/datasett/test/labels'

# Clases
class_names = [
    'Agriculture', 'Airport', 'Beach', 'City', 'Desert', 'Forest', 'Grassland',
    'Highway', 'Lake', 'Mountain', 'Parking', 'Port', 'Railway', 'River'
]
nc = len(class_names)  # Número de clases

# Función para cargar imágenes y etiquetas
def load_data(images_dir, labels_dir, image_size=(128, 128)):
    images = []
    labels = []
    for filename in os.listdir(images_dir):
        img_path = os.path.join(images_dir, filename)
        label_path = os.path.join(labels_dir, filename.replace('.jpg', '.txt'))

        if os.path.exists(label_path):
            # Leer la primera línea del archivo de etiqueta
            with open(label_path, 'r') as file:
                line = file.readline().strip()
                parts = line.split()  # Suponiendo que la etiqueta es el primer valor en la línea
                label = int(parts[0])

            # Cargar la imagen
            img = load_img(img_path, target_size=image_size)
            img = img_to_array(img) / 255.0

            images.append(img)
            labels.append(label)
    return np.array(images), np.array(labels)

# Cargar datos
train_images, train_labels = load_data(train_images_dir, train_labels_dir)
valid_images, valid_labels = load_data(valid_images_dir, valid_labels_dir)
test_images, test_labels = load_data(test_images_dir, test_labels_dir)

# Convertir etiquetas a one-hot encoding
train_labels = to_categorical(train_labels, num_classes=nc)
valid_labels = to_categorical(valid_labels, num_classes=nc)
test_labels = to_categorical(test_labels, num_classes=nc)

# Crear generadores de datos
def data_generator(images, labels, batch_size=32):
    while True:
        for start in range(0, len(images), batch_size):
            end = min(start + batch_size, len(images))
            yield images[start:end], labels[start:end]

train_generator = data_generator(train_images, train_labels)
valid_generator = data_generator(valid_images, valid_labels)
test_generator = data_generator(test_images, test_labels)

# Definición del modelo CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(nc, activation='softmax')
])

# Compilación del modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenamiento del modelo
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_images) // 32,
    validation_data=valid_generator,
    validation_steps=len(valid_images) // 32,
    epochs=10,
    verbose=1
)

# Evaluación del modelo
test_loss, test_acc = model.evaluate(test_generator, steps=len(test_images) // 32)
print(f"\nTest Accuracy: {test_acc*100:.2f}%")

# Predicciones
y_pred = model.predict(test_images)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(test_labels, axis=1)

# Reporte de clasificación
report = classification_report(y_true, y_pred_classes, target_names=class_names)
print(report)

# Matriz de confusión
conf_matrix = confusion_matrix(y_true, y_pred_classes)

# Gráfico de matriz de confusión interactivo
df_cm = pd.DataFrame(conf_matrix, index=class_names, columns=class_names)
fig = px.imshow(df_cm, text_auto=True, aspect="auto", title="Matriz de Confusión")
fig.show()

# Gráficas de rendimiento del modelo
def plot_training_history(history):
    # Gráfico de pérdida
    fig1 = go.Figure()
    fig1.add_trace(go.Scatter(x=list(range(1, len(history.history['loss'])+1)),
                              y=history.history['loss'], mode='lines', name='Loss'))
    fig1.add_trace(go.Scatter(x=list(range(1, len(history.history['val_loss'])+1)),
                              y=history.history['val_loss'], mode='lines', name='Val Loss'))
    fig1.update_layout(title='Evolución de la Pérdida', xaxis_title='Épocas', yaxis_title='Pérdida')

    # Gráfico de precisión
    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(x=list(range(1, len(history.history['accuracy'])+1)),
                              y=history.history['accuracy'], mode='lines', name='Accuracy'))
    fig2.add_trace(go.Scatter(x=list(range(1, len(history.history['val_accuracy'])+1)),
                              y=history.history['val_accuracy'], mode='lines', name='Val Accuracy'))
    fig2.update_layout(title='Evolución de la Precisión', xaxis_title='Épocas', yaxis_title='Precisión')

    # Mostrar los gráficos
    fig1.show()
    fig2.show()

# Mostrar los gráficos de entrenamiento
plot_training_history(history)

# Guardar el modelo entrenado
try:
    model.save('cnn_image_classification_model.h5')
    print("Modelo guardado exitosamente.")
except Exception as e:
    print(f"Error al guardar el modelo: {e}")



Epoch 1/10
21/21 [==============================] - 4s 152ms/step - loss: 2.6265 - accuracy: 0.1592 - val_loss: 2.4502 - val_accuracy: 0.2448
Epoch 2/10
21/21 [==============================] - 3s 136ms/step - loss: 2.2999 - accuracy: 0.2500 - val_loss: 2.1904 - val_accuracy: 0.3095
Epoch 3/10
21/21 [==============================] - 3s 139ms/step - loss: 1.9623 - accuracy: 0.3638 - val_loss: 2.0692 - val_accuracy: 0.2560
Epoch 4/10
21/21 [==============================] - 3s 142ms/step - loss: 1.7990 - accuracy: 0.3982 - val_loss: 2.0163 - val_accuracy: 0.3214
Epoch 5/10
21/21 [==============================] - 3s 134ms/step - loss: 1.5928 - accuracy: 0.4880 - val_loss: 1.7625 - val_accuracy: 0.4107
Epoch 6/10
21/21 [==============================] - 3s 133ms/step - loss: 1.5297 - accuracy: 0.4895 - val_loss: 1.7191 - val_accuracy: 0.4286
Epoch 7/10
21/21 [==============================] - 3s 138ms/step - loss: 1.3342 - accuracy: 0.5524 - val_loss: 1.9688 - val_accuracy: 0.3869
Epoch 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



Modelo guardado exitosamente.
